In [1]:
import pandas as pd
import os

In [2]:
TEXT_EMBEDDING_PATH = "../data/artifacts/google_bert_base_multilingual_cased/google_bert_base_multilingual_cased/bert_base_multilingual_cased.parquet"

In [28]:
SOURCE = "ebnerd_demo"
SPLIT = 'train'

READ_TIME_MIN = 7
READ_TIME_MAX = 600

In [29]:
DATA_PATH = f"../data/{SOURCE}/{SPLIT}/"

In [30]:
df_behave = pd.read_parquet(f"{DATA_PATH}/behaviors.parquet")

In [31]:
df_behave

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,48401,NaN,2023-05-21 21:06:50,21.0,NaN,2,"[9774516, 9771051, 9770028, 9775402, 9774461, ...",[9759966],22779,False,NaN,NaN,NaN,False,21,16.0,27.0
1,152513,9778745.0,2023-05-24 07:31:26,30.0,100.0,1,"[9778669, 9778736, 9778623, 9089120, 9778661, ...",[9778661],150224,False,NaN,NaN,NaN,False,298,2.0,48.0
2,155390,NaN,2023-05-24 07:30:33,45.0,NaN,1,"[9778369, 9777856, 9778500, 9778021, 9778627, ...",[9777856],160892,False,NaN,NaN,NaN,False,401,215.0,100.0
3,214679,NaN,2023-05-23 05:25:40,33.0,NaN,2,"[9776715, 9776406, 9776566, 9776071, 9776808, ...",[9776566],1001055,False,NaN,NaN,NaN,False,1357,40.0,47.0
4,214681,NaN,2023-05-23 05:31:54,21.0,NaN,2,"[9775202, 9776855, 9776688, 9771995, 9776583, ...",[9776553],1001055,False,NaN,NaN,NaN,False,1358,5.0,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24719,579983230,NaN,2023-05-22 08:30:52,35.0,NaN,1,"[9775562, 9775701, 9486486, 9353420, 9775621, ...",[9775562],2053999,False,NaN,NaN,NaN,False,170832,7.0,28.0
24720,579983231,NaN,2023-05-22 08:31:34,89.0,NaN,1,"[9775347, 9678300, 9754160, 9770028, 9775500, ...",[9775361],2053999,False,NaN,NaN,NaN,False,170832,0.0,NaN
24721,579984721,9774541.0,2023-05-22 08:51:33,123.0,100.0,2,"[9775699, 9775722, 9774541, 9775648, 9775325]",[9775699],2060487,False,NaN,NaN,NaN,False,107303,73.0,100.0
24722,579984723,9775699.0,2023-05-22 08:53:36,73.0,100.0,2,"[9772221, 9758424, 9775699, 9775722, 9775621, ...",[9758424],2060487,False,NaN,NaN,NaN,False,107303,101.0,100.0


In [32]:
df_behave = df_behave[["user_id", "impression_time", "article_ids_clicked", "next_read_time"]]

In [33]:
df_behave["article_ids_clicked"].apply(lambda x: len(x)).value_counts()

article_ids_clicked
1    24594
2      114
3        8
4        3
6        3
5        1
7        1
Name: count, dtype: int64

In [34]:
# as the most clicked article is 1, we can take only the first article
df_behave["article_id"] = df_behave["article_ids_clicked"].apply(lambda x: x[0])
df_behave = df_behave.drop(columns=["article_ids_clicked"])

/var/tmp/ipykernel_15373/2801589552.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_behave["article_id"] = df_behave["article_ids_clicked"].apply(lambda x: x[0])


In [35]:
df_behave.columns = ["user_id", "impression_ts", "article_read_time", "article_id"]

In [36]:
# expanding user's history
df_history = pd.read_parquet(f"{DATA_PATH}/history.parquet")

In [37]:
df_history = df_history.explode(['impression_time_fixed', 'scroll_percentage_fixed', 'article_id_fixed', 'read_time_fixed'], ignore_index=True)
df_history = df_history[["user_id", "impression_time_fixed", "article_id_fixed", "read_time_fixed"]]
df_history.columns = ["user_id", "impression_ts", "article_id", "article_read_time"]
df_history

,user_id,impression_ts,article_id,article_read_time
0,13538,2023-04-27 10:17:43,9738663,17.0
1,13538,2023-04-27 10:18:01,9738569,12.0
2,13538,2023-04-27 10:18:13,9738663,4.0
3,13538,2023-04-27 10:18:17,9738490,5.0
4,13538,2023-04-27 10:18:23,9738663,4.0
...,...,...,...,...
258313,2539047,2023-05-09 10:40:37,9755010,5.0
258314,2539047,2023-05-09 10:40:43,9755980,17.0
258315,2539047,2023-05-09 10:41:06,9755980,10.0
258316,2539047,2023-05-09 10:41:17,9755648,3.0


In [38]:
# merging history with the main dataframe
df_behave = pd.concat([df_behave, df_history], ignore_index=True)
df_behave = df_behave.sort_values(by=["user_id", "impression_ts"])
df_behave

,user_id,impression_ts,article_read_time,article_id
205228,11313,2023-04-27 07:18:15,1038.0,9737535
205229,11313,2023-04-27 07:35:35,4.0,9738334
205230,11313,2023-04-27 07:35:39,32.0,9738292
205231,11313,2023-04-27 11:47:11,7.0,9733713
205232,11313,2023-04-28 09:39:38,4.0,9740161
...,...,...,...,...
246403,2589391,2023-05-13 03:42:53,46.0,9762377
246404,2589391,2023-05-18 03:50:09,18.0,9770798
246405,2589391,2023-05-18 03:50:39,2.0,9770594
246406,2589391,2023-05-18 03:50:54,1.0,9770400


In [39]:
# doing some data cleaning

count_before = df_behave.shape[0]

# removing records with low read time
df_behave = df_behave.loc[df_behave["article_read_time"] >= READ_TIME_MIN]
df_behave = df_behave.loc[df_behave["article_read_time"] <= READ_TIME_MAX]

print(f"Removed {count_before - df_behave.shape[0]} records, which is {100 * (count_before - df_behave.shape[0]) / count_before}% of the data")

Removed 87387 records, which is 30.874216547367528% of the data


In [40]:
articles = pd.read_parquet(f"../data/{SOURCE}/articles.parquet")

In [41]:
articles

,article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,...,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
0,3037230,Ishockey-spiller: Jeg troede jeg skulle dø,ISHOCKEY: Ishockey-spilleren Sebastian Harts h...,2023-06-29 06:20:57,False,Ambitionerne om at komme til USA og spille ish...,2003-08-28 08:55:00,None,article_default,https://ekstrabladet.dk/sport/anden_sport/isho...,...,[],"[Kriminalitet, Kendt, Sport, Katastrofe, Mindr...",142,"[327, 334]",sport,NaN,NaN,NaN,0.9752,Negative
1,3044020,Prins Harry tvunget til dna-test,Hoffet tvang Prins Harry til at tage dna-test ...,2023-06-29 06:21:16,False,Den britiske tabloidavis The Sun fortsætter me...,2005-06-29 08:47:00,"[3097307, 3097197, 3104927]",article_default,https://ekstrabladet.dk/underholdning/udlandke...,...,"[PER, PER]","[Kriminalitet, Kendt, Underholdning, Personfar...",414,[432],underholdning,NaN,NaN,NaN,0.7084,Negative
2,3057622,Rådden kørsel på blå plader,Kan ikke straffes: Udenlandske diplomater i Da...,2023-06-29 06:21:24,False,Slingrende spritkørsel. Grove overtrædelser af...,2005-10-10 07:20:00,[3047102],article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,...,[],"[Kriminalitet, Transportmiddel, Bil]",118,[133],nyheder,NaN,NaN,NaN,0.9236,Negative
3,3073151,Mærsk-arvinger i livsfare,FANGET I FLODBØLGEN: Skibsrederens oldebørn må...,2023-06-29 06:21:38,False,To oldebørn af skibsreder Mærsk McKinney Mølle...,2005-01-04 06:59:00,"[3067474, 3067478, 3153705]",article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,...,[],"[Erhverv, Privat virksomhed, Livsstil, Familie...",118,[133],nyheder,NaN,NaN,NaN,0.9945,Negative
4,3193383,Skød svigersøn gennem babydyne,44-årig kvinde tiltalt for drab på ekssvigersø...,2023-06-29 06:22:57,False,En 44-årig mormor blev i dag fremstillet i et ...,2003-09-15 15:30:00,None,article_default,https://ekstrabladet.dk/krimi/article3193383.ece,...,[],"[Kriminalitet, Personfarlig kriminalitet]",140,[],krimi,NaN,NaN,NaN,0.9966,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11772,9803492,Vilde billeder: Vulkan i udbrud i ferieparadis,Der er gang i vulkanen på Hawaiis største ø,2023-06-29 06:49:26,False,Det spyer med lava fra vulkanen Kilauea på Haw...,2023-06-08 05:49:20,"[9803493, 9803494, 9803495, 9803495, 9803494]",article_default,https://ekstrabladet.dk/nyheder/samfund/vilde-...,...,"[LOC, LOC, PER, ORG, ORG]","[Katastrofe, Vejr, Større katastrofe]",118,[133],nyheder,535989.0,100120.0,4112624.0,0.6095,Neutral
11773,9803505,Flyvende Antonsen knuser topspiller,"Verdens nummer syv, Chou Tien-Chen, fik ikke e...",2023-06-29 06:49:26,False,Anders Antonsen har holdt pause fra badmintonb...,2023-06-08 05:54:06,[9803516],article_default,https://ekstrabladet.dk/sport/anden_sport/badm...,...,"[PER, PROD, PER, PER, MISC, MISC, PER, PER, LO...","[Kendt, Begivenhed, Sport, Ketcher- og batspor...",142,"[327, 330]",sport,13320.0,959.0,55691.0,0.8884,Positive
11774,9803525,"Dansk skuespiller: - Jeg nægtede, at jeg var syg",Julie R. Ølgaard fik akut kejsersnit og fødte ...,2023-06-29 06:49:26,False,"Mens hun lå søvnløs, lød kakofonien fra baggår...",2023-06-08 06:45:46,"[9803518, 9803519, 9803520, 9803521, 9803522, ...",article_default,https://ekstrabladet.dk/underholdning/dkkendte...,...,"[PER, PROD, PER, PER, PER, PER, MISC]","[Kendt, Livsstil, Familieliv, Underholdning, F...",414,[425],underholdning,315391.0,50361.0,2550671.0,0.7737,Negative
11775,9803560,Så slemt er det: 14.000 huse er oversvømmet,Tusindvis af huse står under vand i Kherson-re...,2023-06-29 06:49:26,False,Et område på omkring 600 kvadratkilometer står...,2023-06-08 06:25:42,None,article_default,https://ekstrabladet.dk/nyheder/saa-slemt-er-d...,...,"[LOC, LOC, LOC, PROD, PER, LOC, ORG, ORG, LOC]","[International politik, Katastrofe, Større kat...",118,[],nyheder,21318.0,1237.0,67514.0,0

In [42]:
# fill total_read_time with median, where it is missing
articles = articles.fillna(articles["total_read_time"].median())
articles["article_ctr"] = articles["total_pageviews"] / articles["total_inviews"]
# fill ctr with median, where it is missing
articles = articles.fillna(articles["article_ctr"].median())

In [43]:
articles = articles[["article_id", "premium", "article_type", "topics", "category_str", "total_read_time", "sentiment_score", "article_ctr"]]

In [44]:
articles.columns = ["article_id", "article_is_premium", "article_type", "article_topics", "article_category", "article_total_read_time", "article_sentiment", "article_ctr"]

In [45]:
# merging articles with the main dataframe
df_behave = df_behave.merge(articles, on="article_id", how="left")

In [46]:
df_behave

,user_id,impression_ts,article_read_time,article_id,article_is_premium,article_type,article_topics,article_category,article_total_read_time,article_sentiment,article_ctr
0,11313,2023-04-27 07:35:39,32.0,9738292,False,article_default,"[Politik, International politik, Sundhed, Sygd...",nyheder,6637769.0,0.9655,0.213338
1,11313,2023-04-27 11:47:11,7.0,9733713,True,article_standard_feature,"[Samfund, Konflikt og krig, Væbnet konflikt]",nyheder,5907158.0,0.8101,0.134374
2,11313,2023-04-28 11:09:35,34.0,9740174,False,article_default,"[Erhverv, Privat virksomhed, Økonomi]",nyheder,3139645.0,0.8366,0.160523
3,11313,2023-04-28 11:21:02,9.0,9740356,False,article_default,"[Kriminalitet, Personfarlig kriminalitet]",krimi,6991537.0,0.9936,0.215200
4,11313,2023-05-01 06:34:03,323.0,9730301,False,article_default,"[Videnskab, Naturvidenskab]",nyheder,5516137.0,0.6914,0.172012
...,...,...,...,...,...,...,...,...,...,...,...
195650,2589391,2023-05-07 04:21:54,7.0,9752155,False,article_default,"[Kendt, Begivenhed, Sport, Sportsbegivenhed, M...",sport,16268643.0,0.9662,0.215783
195651,2589391,2023-05-12 03:12:29,91.0,9760112,False,article_default,"[Kendt, Underholdning, Film og tv, Begivenhed,...",underholdning,14813650.0,0.9907,0.231048
195652,2589391,2023-05-13 03:42:53,46.0,9762377,False,article_default,"[Transportmiddel, Bil, Katastrofe, Mindre ulykke]",krimi,18644764.0,0.9906,0.257908
195653,2589391,2023-05-18 03:50:09,18.0,9770798,False,article_default,"[Underholdning, Film og tv]",underholdning,8134993.0,0.9341,0.193342


In [47]:
# loading text embeddings
emb_df = pd.read_parquet(TEXT_EMBEDDING_PATH)
emb_df.columns = ["article_id", "article_emb"]

In [48]:
emb_df

,article_id,article_emb
0,3000022,"[-0.35060593, 0.0034366532, -0.1756858, -0.088..."
1,3000063,"[-0.0034481985, 0.22765873, -0.19700234, 0.060..."
2,3000613,"[-0.03811903, -0.030120859, -0.6928362, 0.0801..."
3,3000700,"[-0.29374197, -0.07282147, -0.0926456, -0.0540..."
4,3000840,"[0.0019190352, -0.010633812, -0.49158585, 0.17..."
...,...,...
125536,9803505,"[-0.05083758, 0.068563275, -0.2283888, 0.09498..."
125537,9803510,"[-0.055143625, 0.15021142, -0.15252301, 0.1173..."
125538,9803525,"[-0.035008177, 0.12463321, -0.20443131, 0.1012..."
125539,9803560,"[-0.15544586, 0.07193958, -0.36763963, 0.08102..."


In [49]:
# merging text embeddings with the main dataframe
df_behave = df_behave.merge(emb_df, on="article_id", how="left")
df_behave

,user_id,impression_ts,article_read_time,article_id,article_is_premium,article_type,article_topics,article_category,article_total_read_time,article_sentiment,article_ctr,article_emb
0,11313,2023-04-27 07:35:39,32.0,9738292,False,article_default,"[Politik, International politik, Sundhed, Sygd...",nyheder,6637769.0,0.9655,0.213338,"[-0.13421957, 0.036403418, -0.3091001, 0.08155..."
1,11313,2023-04-27 11:47:11,7.0,9733713,True,article_standard_feature,"[Samfund, Konflikt og krig, Væbnet konflikt]",nyheder,5907158.0,0.8101,0.134374,"[-0.18189247, 0.16488591, -0.23126304, 0.11608..."
2,11313,2023-04-28 11:09:35,34.0,9740174,False,article_default,"[Erhverv, Privat virksomhed, Økonomi]",nyheder,3139645.0,0.8366,0.160523,"[-0.11109238, -0.045725856, -0.19442482, 0.083..."
3,11313,2023-04-28 11:21:02,9.0,9740356,False,article_default,"[Kriminalitet, Personfarlig kriminalitet]",krimi,6991537.0,0.9936,0.215200,"[-0.10872016, 0.06527501, -0.26191005, 0.10563..."
4,11313,2023-05-01 06:34:03,323.0,9730301,False,article_default,"[Videnskab, Naturvidenskab]",nyheder,5516137.0,0.6914,0.172012,"[-0.012549153, 0.089903, -0.1078802, 0.0717587..."
...,...,...,...,...,...,...,...,...,...,...,...,...
195650,2589391,2023-05-07 04:21:54,7.0,9752155,False,article_default,"[Kendt, Begivenhed, Sport, Sportsbegivenhed, M...",sport,16268643.0,0.9662,0.215783,"[-0.20416486, 0.118332416, -0.23212367, 0.1696..."
195651,2589391,2023-05-12 03:12:29,91.0,9760112,False,article_default,"[Kendt, Underholdning, Film og tv, Begivenhed,...",underholdning,14813650.0,0.9907,0.231048,"[-0.036725495, 0.14778818, -0.20494908, 0.1472..."
195652,2589391,2023-05-13 03:42:53,46.0,9762377,False,article_default,"[Transportmiddel, Bil, Katastrofe, Mindre ulykke]",krimi,18644764.0,0.9906,0.257908,"[-0.3430326, 0.052118003, -0.49910527, 0.07473..."
195653,2589391,2023-05-18 03:50:09,18.0,9770798,False,article_default,"[Underholdning, Film og tv]",underholdning,8134993.0,0.9341,0.193342,"[-0.15496331, 0.057488672, -0.22700496, 0.0563..."


In [50]:
# writing the final dataframe
os.makedirs(f"../data/{SOURCE}_modified/", exist_ok=True)

df_behave.to_parquet(f"../data/{SOURCE}_modified/{SPLIT}.parquet", index=False)

In [51]:
df_behave

,user_id,impression_ts,article_read_time,article_id,article_is_premium,article_type,article_topics,article_category,article_total_read_time,article_sentiment,article_ctr,article_emb
0,11313,2023-04-27 07:35:39,32.0,9738292,False,article_default,"[Politik, International politik, Sundhed, Sygd...",nyheder,6637769.0,0.9655,0.213338,"[-0.13421957, 0.036403418, -0.3091001, 0.08155..."
1,11313,2023-04-27 11:47:11,7.0,9733713,True,article_standard_feature,"[Samfund, Konflikt og krig, Væbnet konflikt]",nyheder,5907158.0,0.8101,0.134374,"[-0.18189247, 0.16488591, -0.23126304, 0.11608..."
2,11313,2023-04-28 11:09:35,34.0,9740174,False,article_default,"[Erhverv, Privat virksomhed, Økonomi]",nyheder,3139645.0,0.8366,0.160523,"[-0.11109238, -0.045725856, -0.19442482, 0.083..."
3,11313,2023-04-28 11:21:02,9.0,9740356,False,article_default,"[Kriminalitet, Personfarlig kriminalitet]",krimi,6991537.0,0.9936,0.215200,"[-0.10872016, 0.06527501, -0.26191005, 0.10563..."
4,11313,2023-05-01 06:34:03,323.0,9730301,False,article_default,"[Videnskab, Naturvidenskab]",nyheder,5516137.0,0.6914,0.172012,"[-0.012549153, 0.089903, -0.1078802, 0.0717587..."
...,...,...,...,...,...,...,...,...,...,...,...,...
195650,2589391,2023-05-07 04:21:54,7.0,9752155,False,article_default,"[Kendt, Begivenhed, Sport, Sportsbegivenhed, M...",sport,16268643.0,0.9662,0.215783,"[-0.20416486, 0.118332416, -0.23212367, 0.1696..."
195651,2589391,2023-05-12 03:12:29,91.0,9760112,False,article_default,"[Kendt, Underholdning, Film og tv, Begivenhed,...",underholdning,14813650.0,0.9907,0.231048,"[-0.036725495, 0.14778818, -0.20494908, 0.1472..."
195652,2589391,2023-05-13 03:42:53,46.0,9762377,False,article_default,"[Transportmiddel, Bil, Katastrofe, Mindre ulykke]",krimi,18644764.0,0.9906,0.257908,"[-0.3430326, 0.052118003, -0.49910527, 0.07473..."
195653,2589391,2023-05-18 03:50:09,18.0,9770798,False,article_default,"[Underholdning, Film og tv]",underholdning,8134993.0,0.9341,0.193342,"[-0.15496331, 0.057488672, -0.22700496, 0.0563..."


In [52]:
# additionally join both train and validation data
df_behave_train = pd.read_parquet(f"../data/{SOURCE}_modified/train.parquet")
df_behave_val = pd.read_parquet(f"../data/{SOURCE}_modified/validation.parquet")

df_behave = pd.concat([df_behave_train, df_behave_val], ignore_index=True)
df_behave.to_parquet(f"../data/{SOURCE}_modified/all.parquet", index=False)